__Required files:__
* ##### Part 1
    `gs://phenotype_31063/hail/imputed/ukb31063.dosage.autosomes.mt`

    `gs://nbaya/split/hapmap3_variants.tsv`

    `gs://nbaya/split/hapmap3_variants_v2.tsv`

    `gs://phenotype_31063/ukb31063.gwas_covariates.both_sexes.tsv`

* ##### Part 2
    `gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_v1.mt` **(Deprecated)**
    `gs://nbaya/split/ukb31063.hm3_variants.gwas_samples.mt` (Current)
* ##### Part 3
    `gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_v2.mt` **(Deprecated)**
    `gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_repart.mt` (Current)
    `gs://nbaya/split/meta_split/ukb31063.hm3_variants.gwas_samples_+phen+desc+.tsv`
* ##### Part 4
    `gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_+phen+_grouped_v1.mt`
* ##### Part 5
    `gs://nbaya/split/meta_split/ukb31063.hm3_+phen+_gmt.mt`

<br>

__Final output:__ 
`gs://nbaya/split/meta_split/ukb31063.hm3_+phen+_meta_split_final_s2.tsv.bgz`

__Note:__
The latest version of this is now a Python script submitted as a job to the cloud. When using this notebook, make sure to check code with the latest version to ensure that everything matches (most importantly, file names).

### Import packages, set phenotype 

In [2]:
import hail as hl
import numpy as np
import pandas as pd

phen_dict = {
    '50':'height',
    '20160':'smoking',
    '2443':'diabetes',
    '23107':'legimp_r',
    '6138_100':'qual_none'
}

phen = '50'
desc = phen_dict[phen]
batch = '1'
n_chunks = 300
variant_set = 'hm3'
constant_sex_ratio = True

# Part 1
Create filtered matrix table annotated with locus and alleles (Done)

In [3]:
mt = hl.read_matrix_table('gs://phenotype_31063/hail/imputed/ukb31063.dosage.autosomes.mt')


Initializing Spark and Hail with default parameters...
Running on Apache Spark version 2.2.1
SparkUI available at http://10.128.0.29:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2-e60bdb1a125a
LOGGING: writing to /home/hail/hail-20181202-2019-0.2-e60bdb1a125a.log


In [4]:
mt.count()

(93095623, 487409)

In [2]:
if variant_set is 'hm3':
        variants = hl.import_table('gs://nbaya/split/hapmap3_variants.tsv')
elif variant_set is 'qc_pos':
        variants = hl.import_table('gs://ukb31063-mega-gwas/qc/ukb31063.gwas_variants.autosomes.tsv')
variants = variants.annotate(**hl.parse_variant(variants.v))
variants = variants.key_by('locus','alleles')

Initializing Spark and Hail with default parameters...
Running on Apache Spark version 2.2.1
SparkUI available at http://10.128.0.6:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2-e60bdb1a125a
LOGGING: writing to /home/hail/hail-20181204-1257-0.2-e60bdb1a125a.log
2018-12-04 12:58:06 Hail: INFO: Reading table with no type imputation
  Loading column 'v' as type 'str' (type not specified)



In [3]:
variants

In [ ]:
# variants.write('gs://nbaya/split/hapmap3_variants_v2.tsv')

In [2]:
variants = hl.import_table('gs://nbaya/split/'+variant_set+'_variants.ht') # for hm3: import table hapmap3_variants.tsv'

mt = hl.read_matrix_table('gs://phenotype_31063/hail/imputed/ukb31063.dosage.autosomes.mt')
mt = mt.filter_rows(hl.is_defined(variants[mt.locus,mt.alleles])) #filter to variant_set variants

covs = hl.import_table('gs://phenotype_31063/ukb31063.gwas_covariates.both_sexes.tsv',
                             key='s', impute=True, types={'s': hl.tstr})

mt = mt.annotate_cols(**covs[mt.s])
mt = mt.filter_cols(hl.is_defined(mt.PC1), keep=True)

Initializing Spark and Hail with default parameters...
Running on Apache Spark version 2.2.1
SparkUI available at http://10.128.15.248:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2-e60bdb1a125a
LOGGING: writing to /home/hail/hail-20181204-1330-0.2-e60bdb1a125a.log


FatalError: FileNotFoundException: Item not found: nbaya/split/qc_pos_variants.ht

Java stack trace:
java.io.FileNotFoundException: Item not found: nbaya/split/qc_pos_variants.ht
	at com.google.cloud.hadoop.gcsio.GoogleCloudStorageExceptions.getFileNotFoundException(GoogleCloudStorageExceptions.java:41)
	at com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.open(GoogleCloudStorageImpl.java:629)
	at com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystem.open(GoogleCloudStorageFileSystem.java:343)
	at com.google.cloud.hadoop.fs.gcs.GoogleHadoopFSInputStream.<init>(GoogleHadoopFSInputStream.java:121)
	at com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystemBase.open(GoogleHadoopFileSystemBase.java:1188)
	at org.apache.hadoop.fs.FileSystem.open(FileSystem.java:787)
	at is.hail.utils.richUtils.RichHadoopConfiguration$.is$hail$utils$richUtils$RichHadoopConfiguration$$open$extension(RichHadoopConfiguration.scala:37)
	at is.hail.utils.richUtils.RichHadoopConfiguration$.readFile$extension

In [5]:
hl.read_table('gs://nbaya/split/qc_pos_variants.ht/')

Filter to HapMap 3 variants

In [7]:
mt = mt.filter_rows(hl.is_defined(variants[mt.locus,mt.alleles])) 

In [ ]:
mt.count()

2018-12-02 20:22:45 Hail: INFO: Ordering unsorted dataset with network shuffle


In [ ]:
ht_samples = hl.import_table('gs://phenotype_31063/ukb31063.gwas_covariates.both_sexes.tsv',
                             key='s', impute=True, types={'s': hl.tstr})


Annotate with covariates

In [ ]:
mt = mt.annotate_cols(**ht_samples[mt.s])
mt = mt.filter_cols(hl.is_defined(mt.PC1), keep=True) #Only keeps columns with PC1 defined

In [ ]:
# mt.write('gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_v1.mt', overwrite=True) #Deprecated
mt.write('gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_prerepart.mt', overwrite=True)

# Part 2
Repartition (Done)
#### Note: Task requires shuffle reading and writing. Do not use preemptible workers.

In [ ]:
# mt = hl.read_matrix_table('gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_v1.mt') #Deprecated
mt = hl.read_matrix_table('gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_prerepart.mt')


mt = mt.repartition(1000)

# mt.write('gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_v2.mt', overwrite=True) #Deprecated
mt.write('gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_repart.mt', overwrite=True)


# Part 3
Annotate with phenotype and label with group IDs

In [3]:
mt0 = hl.read_matrix_table(f'gs://nbaya/split/ukb31063.{variant_set}_variants.gwas_samples_repart.mt')

if 'sim' in phen:
    print('\nReading simulated phenotype...')
    if variant_set == 'qc_pos':
        mt1 = hl.read_matrix_table('gs://nbaya/rg_sex/qc_pos.50_sim_inf_h2_0.485223.mt') #outdated
    elif variant_set == 'hm3':
#            mt1 = hl.read_matrix_table('gs://nbaya/rg_sex/50_sim_inf_h2_0.485223.mt')      
        sim_phen = phen.split('_sim')[0]
        if sim_phen == '50':
            phen_tb = hl.read_table(f'gs://nbaya/ldscsim/{variant_set}.phen_{sim_phen}.sim_h2_{0.485223}.ht')

else:
    print('\nReading UKB phenotype...')
#        mt0 = hl.read_matrix_table('gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_v2.mt') #old version

    if phen == '50_raw':
        phen_tb0 = hl.import_table('gs://nbaya/ukb31063.50_raw.tsv.bgz',missing='',impute=True,types={'s': hl.tstr}).rename({phen: 'phen'})
    elif phen == '50_raw_res':
        phen_tb0 = hl.read_table('gs://nbaya/split/50_raw_linreg.ht').rename({'res': 'phen'})
    else:
        phen_tb0 = hl.import_table('gs://phenotype_31063/ukb31063.phesant_phenotypes.both_sexes.tsv.bgz',
                                      missing='',impute=True,types={'"userId"': hl.tstr}).rename({ '"userId"': 's', '"'+phen+'"': 'phen'})
        phen_tb0 = phen_tb0.key_by('s')
    phen_tb = phen_tb0.select(phen_tb0['phen'])    

Initializing Spark and Hail with default parameters...
Running on Apache Spark version 2.2.3
SparkUI available at http://10.128.0.29:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.12-13681278eb89
LOGGING: writing to /home/hail/hail-20190411-1153-0.2.12-13681278eb89.log



Reading UKB phenotype...


2019-04-11 11:53:36 Hail: INFO: Reading table to impute column types
2019-04-11 11:54:44 Hail: INFO: Finished type imputation
  Loading column '"userId"' as type 'str' (user-specified)
  Loading column '"46"' as type 'float64' (imputed)
  Loading column '"47"' as type 'float64' (imputed)
  Loading column '"48"' as type 'float64' (imputed)
  Loading column '"49"' as type 'float64' (imputed)
  Loading column '"50"' as type 'float64' (imputed)
  Loading column '"51"' as type 'float64' (imputed)
  Loading column '"78"' as type 'float64' (imputed)
  Loading column '"84"' as type 'float64' (imputed)
  Loading column '"93"' as type 'float64' (imputed)
  Loading column '"94"' as type 'float64' (imputed)
  Loading column '"102"' as type 'float64' (imputed)
  Loading column '"129"' as type 'float64' (imputed)
  Loading column '"130"' as type 'float64' (imputed)
  Loading column '"134"' as type 'str' (imputed)
  Loading column '"135"' as type 'str' (imputed)
  Loading column '"136"' as type 'str'

  Loading column '"100001"' as type 'float64' (imputed)
  Loading column '"100002"' as type 'float64' (imputed)
  Loading column '"100003"' as type 'float64' (imputed)
  Loading column '"100004"' as type 'float64' (imputed)
  Loading column '"100005"' as type 'float64' (imputed)
  Loading column '"100006"' as type 'float64' (imputed)
  Loading column '"100007"' as type 'float64' (imputed)
  Loading column '"100008"' as type 'float64' (imputed)
  Loading column '"100009"' as type 'float64' (imputed)
  Loading column '"100010"' as type 'str' (imputed)
  Loading column '"100011"' as type 'float64' (imputed)
  Loading column '"100012"' as type 'float64' (imputed)
  Loading column '"100013"' as type 'float64' (imputed)
  Loading column '"100014"' as type 'float64' (imputed)
  Loading column '"100015"' as type 'float64' (imputed)
  Loading column '"100016"' as type 'float64' (imputed)
  Loading column '"100017"' as type 'float64' (imputed)
  Loading column '"100018"' as type 'float64' (imput

In [4]:
mt1 = mt0.annotate_cols(phen_str = hl.str(phen_tb[mt0.s]['phen']).replace('\"',''))
mt1 = mt1.filter_cols(mt1.phen_str == '',keep=False)

if phen_tb.phen.dtype == hl.dtype('bool'):
    mt1 = mt1.annotate_cols(phen = hl.bool(mt1.phen_str)).drop('phen_str')
else:
    mt1 = mt1.annotate_cols(phen = hl.float64(mt1.phen_str)).drop('phen_str')    

In [5]:
#Remove withdrawn samples
withdrawn = hl.import_table('gs://nbaya/w31063_20181016.csv',missing='',no_header=True)
withdrawn_set = set(withdrawn.f0.take(withdrawn.count()))
mt1 = mt1.filter_cols(hl.literal(withdrawn_set).contains(mt1['s']),keep=False) 
mt1 = mt1.key_cols_by('s')

n_samples = mt1.count_cols()
print(f'\n>>> N samples = {n_samples} <<<') #expect n samples to match n_non_missing from phenotypes.both_sexes.tsv, minus withdrawn samples.

2019-04-11 11:54:46 Hail: INFO: Reading table with no type imputation
  Loading column 'f0' as type 'str' (type not specified)




>>> N samples = 360338 <<<


### Approach 1 (does not work)

In [19]:
mt2 = mt1.add_col_index()
n_per_sex = np.asarray([mt2.filter_cols(mt2.isFemale==i).count_cols() for i in range(2)])
print(n_per_sex)

In [24]:
group_size = (n_per_sex/n_chunks).astype(int)+1

In [49]:
randstate = np.random.RandomState(int(batch)) #seed with batch number
group_ids = []
for i in range(2):
    temp=np.ndarray.tolist(np.ndarray.flatten(np.asarray([range(n_chunks)]*group_size[i]))) 
    temp=temp[0:n_per_sex[i]]
    randstate.shuffle(temp)
    group_ids.append(temp)


In [39]:
cols = mt2.cols().key_by('s')

2019-04-08 16:03:29 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


In [43]:
cols = cols.order_by(cols.isFemale)

In [56]:
group_ids_all = [i for x in group_ids for i in x] 
cols = cols.add_index().key_by('idx')
cols = cols.annotate(group_id = hl.literal(group_ids_all)[hl.int32(cols.idx)])

In [55]:
group_ids_all

[51,
 23,
 68,
 18,
 143,
 135,
 295,
 258,
 141,
 203,
 147,
 130,
 184,
 44,
 151,
 254,
 233,
 40,
 227,
 25,
 240,
 257,
 109,
 246,
 255,
 95,
 183,
 283,
 121,
 97,
 253,
 276,
 83,
 54,
 170,
 19,
 295,
 85,
 52,
 272,
 74,
 32,
 280,
 194,
 5,
 124,
 100,
 156,
 258,
 235,
 209,
 299,
 13,
 93,
 20,
 43,
 60,
 212,
 266,
 140,
 262,
 24,
 28,
 64,
 79,
 251,
 142,
 118,
 196,
 109,
 63,
 144,
 165,
 89,
 53,
 198,
 73,
 176,
 280,
 115,
 45,
 98,
 65,
 58,
 11,
 199,
 4,
 119,
 137,
 262,
 122,
 13,
 20,
 147,
 135,
 147,
 216,
 165,
 176,
 228,
 242,
 44,
 143,
 201,
 77,
 185,
 104,
 261,
 111,
 225,
 275,
 34,
 52,
 123,
 136,
 227,
 109,
 262,
 293,
 40,
 62,
 49,
 198,
 41,
 91,
 161,
 194,
 289,
 238,
 137,
 23,
 72,
 105,
 193,
 193,
 168,
 46,
 262,
 235,
 60,
 99,
 210,
 249,
 36,
 231,
 129,
 60,
 184,
 121,
 58,
 165,
 78,
 282,
 151,
 74,
 115,
 197,
 212,
 41,
 42,
 111,
 149,
 21,
 174,
 69,
 71,
 238,
 272,
 110,
 16,
 242,
 72,
 275,
 138,
 151,
 149,
 144,
 288

In [57]:
cols.show()

2019-04-08 17:15:28 Hail: INFO: Coerced sorted dataset
2019-04-08 17:15:32 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-04-08 17:15:47 Hail: INFO: Coerced sorted dataset
2019-04-08 17:18:18 Hail: INFO: Coerced sorted dataset
2019-04-08 17:18:24 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-04-08 17:18:38 Hail: INFO: Coerced sorted dataset


s,isFemale,age,age_squared,age_isFemale,age_squared_isFemale,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,phen,col_idx,idx,group_id
str,bool,int32,int32,int32,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int32
"""1000019""",true,63,3969,63,3969,-1.58e-02,6.21e-03,7.21e-03,4.78e-03,-4.36e-03,1.35e-03,6.03e-04,8.67e-04,1.56e-03,1.51e-03,-7.76e-03,1.88e-03,7.77e-03,3.65e-03,4.57e-03,6.79e-03,-3.51e-04,3.70e-04,-1.39e-03,1.08e-02,-4.77e-01,192240,0,51
"""1000022""",false,65,4225,0,0,7.68e-03,3.66e-03,-1.15e-02,-8.13e-03,3.13e-03,1.18e-02,-5.00e-03,-4.53e-03,-3.86e-03,2.07e-04,9.97e-03,2.40e-03,-1.06e-03,2.86e-03,5.93e-03,3.00e-03,4.17e-03,-7.52e-04,-6.81e-03,-5.51e-03,1.26e+00,308589,1,23
"""1000035""",false,59,3481,0,0,-7.50e-03,3.38e-03,6.34e-03,-1.43e-04,3.42e-03,2.55e-03,-5.29e-03,-5.35e-03,3.80e-03,2.10e-03,8.38e-03,-5.25e-03,-2.46e-03,7.08e-03,2.66e-03,-2.51e-03,1.54e-02,3.09e-03,3.84e-03,6.23e-03,1.04e+00,10011,2,68
"""1000046""",true,57,3249,57,3249,-1.85e-02,3.90e-03,2.80e-03,5.85e-03,-5.98e-03,-2.84e-03,-1.97e-03,1.34e-03,3.38e-03,-6.33e-03,-3.45e-03,-1.89e-03,3.24e-03,7.34e-03,2.46e-03,-1.06e-04,1.18e-02,-4.30e-03,-1.67e-03,-3.92e-03,-2.15e-01,179958,3,18
"""1000054""",false,44,1936,0,0,1.14e-02,2.92e-03,2.99e-03,-1.82e-03,2.37e-03,5.24e-03,6.95e-03,-6.79e-04,2.12e-03,7.16e-03,5.31e-03,-1.05e-03,1.54e-03,9.58e-03,-4.18e-04,2.08e-03,-2.47e-03,7.08e-03,-1.82e-03,1.14e-03,1.07e+00,106273,4,143
"""1000063""",false,65,4225,0,0,-1.64e-02,4.56e-03,-8.76e-04,-3.40e-04,-1.84e-03,-7.01e-03,-1.16e-02,1.31e-02,-7.50e-03,2.17e-03,2.62e-03,6.22e-03,-1.49e-03,8.74e-04,3.92e-03,-1.03e-02,-1.53e-03,-7.56e-03,-4.59e-03,6.80e-03,1.27e+00,267388,5,135
"""1000081""",false,66,4356,0,0,2.61e-02,7.57e-03,-7.58e-03,-1.18e-02,5.58e-05,1.48e-05,1.28e-02,1.23e-03,-2.91e-03,-1.80e-03,-6.18e-03,-1.05e-02,-6.11e-03,2.28e-03,-2.02e-04,6.02e-04,-2.75e-03,2.17e-03,2.29e-04,-3.10e-03,8.73e-01,140345,6,295
"""1000090""",false,54,2916,0,0,-1.01e-02,-7.00e-03,-2.15e-03,4.91e-04,2.01e-03,-1.24e-03,1.03e-03,-3.98e-03,1.07e-02,-6.53e-03,1.59e-02,1.57e-03,-1.53e-03,-1.68e-03,-8.91e-04,5.23e-03,-1.08e-02,7.58e-03,-1.13e-02,6.64e-03,1.27e-01,152165,7,258
"""1000105""",true,64,4096,64,4096,-1.27e-02,1.01e-02,2.05e-02,-1.47e-03,-1.28e-02,1.00e-02,6.28e-04,5.75e-03,-2.68e-03,-8.31e-03,1.99e-03,-2.82e-03,-5.14e-03,-2.86e-03,2.30e-03,4.80e-03,3.38e-03,-3.45e-03,5.54e-03,1.43e-02,-2.30e+00,339478,8,141


In [58]:
cols = cols.key_by('s')
mt3 = mt2.annotate_cols(group_id = cols[mt2.s].group_id)

In [59]:
mt3.aggregate_cols(hl.agg.group_by(mt3.group_id,hl.agg.mean(mt3.isFemale)))

2019-04-08 17:20:35 Hail: INFO: Coerced sorted dataset
2019-04-08 17:20:39 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-04-08 17:20:54 Hail: INFO: Coerced sorted dataset
2019-04-08 17:22:45 Hail: INFO: Coerced sorted dataset


{69: 0.5158069883527454,
 138: 0.5237302248126561,
 101: 0.5328892589508742,
 249: 0.5403830141548709,
 234: 0.5378850957535387,
 0: 0.519134775374376,
 88: 0.5154038301415487,
 170: 0.5270607826810991,
 115: 0.5212323064113239,
 217: 0.533721898417985,
 276: 0.555,
 5: 0.5158069883527454,
 120: 0.5495420482930891,
 247: 0.5378850957535387,
 269: 0.5291666666666667,
 202: 0.5295587010824313,
 10: 0.5449251247920133,
 56: 0.5415973377703827,
 142: 0.5253955037468776,
 153: 0.5428809325562032,
 174: 0.5395503746877602,
 185: 0.5195670274771024,
 42: 0.5133111480865225,
 24: 0.5266222961730449,
 288: 0.5341666666666667,
 37: 0.5332778702163061,
 25: 0.5607321131447587,
 257: 0.56,
 52: 0.5557404326123128,
 14: 0.5349417637271214,
 184: 0.5528726061615321,
 110: 0.5129059117402165,
 125: 0.5445462114904246,
 196: 0.5395503746877602,
 157: 0.537052456286428,
 189: 0.5453788509575354,
 20: 0.5133111480865225,
 46: 0.5266222961730449,
 93: 0.5278934221482098,
 284: 0.5291666666666667,
 152: 0

### Approach 2

In [6]:
mt_ls = [mt1.filter_cols(mt1.isFemale==0), mt1.filter_cols(mt1.isFemale==1)]

In [7]:
mt_final = []
for mt_temp in mt_ls:
    n_samples = mt_temp.count_cols()
    print('\n>>> N samples = '+str(n_samples)+' <<<') #expect n samples to match n_non_missing from phenotypes.both_sexes.tsv, minus withdrawn samples.
    
    mt_temp2 = mt_temp.add_col_index()
    group_size = int(n_samples/n_chunks)+1     #the ideal number of samples in each group
    #list of group ids to be paired to each sample (Note: length of group_ids > # of cols in mt, but it doesn't affect the result)
    group_ids = np.ndarray.tolist(np.ndarray.flatten(np.asarray([range(n_chunks)]*group_size))) 
    group_ids = group_ids[0:n_samples]
    randstate = np.random.RandomState(int(batch)) #seed with batch number
    randstate.shuffle(group_ids)
    mt_final.append(mt_temp2.annotate_cols(group_id = hl.literal(group_ids)[hl.int32(mt_temp2.col_idx)])) #assign group ids # OLD VERSION

mt3 = mt_final[0].union_cols(mt_final[1])


>>> N samples = 166583 <<<

>>> N samples = 193755 <<<


In [8]:
mt3.aggregate_cols(hl.agg.group_by(mt3.group_id,hl.agg.mean(mt3.isFemale)))

2019-04-11 12:02:30 Hail: INFO: Coerced sorted dataset


{69: 0.5374376039933444,
 138: 0.5378850957535387,
 101: 0.5378850957535387,
 249: 0.5378850957535387,
 234: 0.5378850957535387,
 0: 0.5374376039933444,
 88: 0.5378850957535387,
 170: 0.5378850957535387,
 115: 0.5378850957535387,
 217: 0.5378850957535387,
 276: 0.5375,
 5: 0.5374376039933444,
 120: 0.5378850957535387,
 247: 0.5378850957535387,
 269: 0.5375,
 202: 0.5378850957535387,
 10: 0.5374376039933444,
 56: 0.5374376039933444,
 142: 0.5378850957535387,
 153: 0.5378850957535387,
 174: 0.5378850957535387,
 185: 0.5378850957535387,
 42: 0.5374376039933444,
 24: 0.5374376039933444,
 288: 0.5375,
 37: 0.5374376039933444,
 25: 0.5374376039933444,
 257: 0.5375,
 52: 0.5374376039933444,
 14: 0.5374376039933444,
 184: 0.5378850957535387,
 110: 0.5378850957535387,
 125: 0.5378850957535387,
 196: 0.5378850957535387,
 157: 0.5378850957535387,
 189: 0.5378850957535387,
 20: 0.5374376039933444,
 46: 0.5374376039933444,
 93: 0.5378850957535387,
 284: 0.5375,
 152: 0.5378850957535387,
 228: 0.537

In [20]:
group_size = int(n_samples/n_chunks)+1     #the ideal number of samples in each group
#list of group ids to be paired to each sample (Note: length of group_ids > # of cols in mt, but it doesn't affect the result)
group_ids = np.ndarray.tolist(np.ndarray.flatten(np.asarray([range(n_chunks)]*group_size))) 
group_ids = group_ids[0:n_samples]
randstate = np.random.RandomState(int(batch)) #seed with batch number
randstate.shuffle(group_ids)
mt3 = mt2.annotate_cols(group_id = hl.literal(group_ids)[hl.int32(mt2.col_idx)]) #assign group ids # OLD VERSION
ht_group_ids = mt3.select_cols(mt3.group_id).cols() #assign group ids

[166583, 193755]


In [63]:
# Takes about 20 min with 100 workers, 0 preemptibles
# Requires shuffle reading and writing
mt3.write(f'gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_{phen}_grouped_v1_new.mt', overwrite=True)


2018-10-02 16:24:56 Hail: INFO: Coerced sorted dataset
2018-10-02 17:16:30 Hail: INFO: wrote 1089172 items in 1000 partitions to gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_50_grouped_v1_new.mt


# Part 4
Run linear regression (Done)

In [2]:
mt = hl.read_matrix_table('gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_'+phen+'_grouped_batch_'+batch+'.mt')


Initializing Spark and Hail with default parameters...
Running on Apache Spark version 2.2.1
SparkUI available at http://10.128.0.56:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version devel-654043004581
NOTE: This is a beta version. Interfaces may change
  during the beta period. We recommend pulling
  the latest changes weekly.
LOGGING: writing to /home/hail/hail-20181002-1521-devel-654043004581.log


In [ ]:
mt = mt.rename({'dosage': 'x', 'phen': 'y'})

cov_list = [ mt['isFemale'], mt['age'], mt['age_squared'], mt['age_isFemale'],
            mt['age_squared_isFemale'] ]+ [mt['PC{:}'.format(i)] for i in range(1, 21)] 

In [ ]:
gmt = (mt.group_cols_by(mt.group_id)
        .aggregate(linreg = hl.agg.linreg(y=mt.y, x = [mt.x, 1] + cov_list)))

In [ ]:
# Takes about 1.7 hours (with 10 workers + 200 pre-emptible workers)
# First and only stage should have 1000 tasks
gmt.select_entries(beta=gmt.linreg.beta[0],standard_error=gmt.linreg.standard_error[0]).write('gs://nbaya/split/meta_split/ukb31063.hm3_'+phen+'_gmt_test.mt')


### Check linear regression results
Used only when constant_sex_ratio = True

In [12]:
gmt = hl.read_matrix_table(f'gs://nbaya/split/meta_split/ukb31063.{variant_set}_{phen}_gmt{n_chunks}_constantsexratio_{constant_sex_ratio}_batch_{batch}.mt') #used for n=150 and other phenotypes
gmt = gmt.add_row_index()
gmt = gmt.key_rows_by('row_idx')
gmt = gmt.drop('locus','alleles','varid')
gmt = gmt.rename({'rsid': 'SNP'})

In [11]:
gmt.beta.show()

+---------------+------------+----------+-----------+
| locus         | alleles    | group_id |      beta |
+---------------+------------+----------+-----------+
| locus<GRCh37> | array<str> |    int32 |   float64 |
+---------------+------------+----------+-----------+
| 1:754182      | ["A","G"]  |        0 |  1.82e-02 |
| 1:754182      | ["A","G"]  |        1 |  5.75e-02 |
| 1:754182      | ["A","G"]  |        2 |  7.73e-02 |
| 1:754182      | ["A","G"]  |        3 | -4.39e-02 |
| 1:754182      | ["A","G"]  |        4 |  6.18e-02 |
| 1:754182      | ["A","G"]  |        5 | -2.48e-02 |
| 1:754182      | ["A","G"]  |        6 | -3.53e-02 |
| 1:754182      | ["A","G"]  |        7 | -3.22e-02 |
| 1:754182      | ["A","G"]  |        8 | -9.15e-02 |
| 1:754182      | ["A","G"]  |        9 |  4.98e-02 |
+---------------+------------+----------+-----------+
showing top 10 rows



# Part 5
Split 300 subgroups into two groups

In [3]:
gmt = hl.read_matrix_table('gs://nbaya/split/meta_split/ukb31063.hm3_'+phen+'_gmt.mt')

gmt = gmt.add_row_index()
gmt = gmt.key_rows_by('row_idx')
gmt = gmt.drop('locus','alleles','varid')
gmt = gmt.rename({'rsid': 'SNP'})

2018-09-19 20:25:25 Hail: INFO: Coerced sorted dataset


In [13]:
n = 300 #number of subgroups
pi = ['A']*int(n/2) + ['B']*int(n/2)

In [14]:
np.random.shuffle(pi)
gmt = gmt.add_col_index()
gmt = gmt.annotate_cols(label = hl.literal(pi)[hl.int32(gmt.col_idx)])


In [15]:
# print(gmt.aggregate_cols(hl.agg.counter(gmt.label))) #determine the evenness of the split
mt = gmt.group_cols_by(gmt.label).aggregate(unnorm_meta_beta=hl.agg.sum(gmt.beta / gmt.standard_error ** 2), inv_se2 = hl.agg.sum(1 / gmt.standard_error ** 2))


In [ ]:
#Make table of just the rsid column (already complete)
# rsid_mt = mt.drop( 'unnorm_meta_beta', 'inv_se2')
# rsid_tb = rsids_mt.make_table()
# rsid_tb.show()
# rsid_tb.export('gs://nbaya/split/meta_split/meta_split_rsid.tsv.bgz')


In [16]:
ht = mt.make_table()
# ht = ht.annotate(A_meta_beta = ht['A.unnorm_meta_beta'] / ht['A.inv_se2'],
#                    A_meta_se = hl.sqrt(1 / ht['A.inv_se2']), 
#                    B_meta_beta = ht['B.unnorm_meta_beta'] / ht['B.inv_se2'], 
#                    B_meta_se = hl.sqrt(1 / ht['B.inv_se2']))
ht = ht.annotate(A_Z = ht['A.unnorm_meta_beta'] / hl.sqrt(ht['A.inv_se2']),
                     B_Z = ht['B.unnorm_meta_beta'] / hl.sqrt(ht['B.inv_se2']))
ht = ht.drop('A.unnorm_meta_beta','B.unnorm_meta_beta','A.inv_se2','B.inv_se2').key_by('SNP')
ht.show()

2019-04-11 12:06:46 Hail: INFO: Coerced sorted dataset
2019-04-11 12:07:09 Hail: INFO: Coerced sorted dataset
2019-04-11 12:07:26 Hail: INFO: Coerced sorted dataset


SNP,row_idx,A_Z,B_Z
str,int64,float64,float64
"""rs1000000""",798306,-1.48e+00,4.33e-01
"""rs10000010""",267854,1.23e+00,1.92e+00
"""rs1000002""",252333,3.25e-01,1.43e+00
"""rs10000023""",292853,-8.11e-01,5.49e-01
"""rs1000003""",220949,-1.01e+00,-1.41e+00
"""rs10000033""",307480,-1.10e+00,4.75e-01
"""rs10000037""",274436,3.29e+00,2.58e+00
"""rs10000041""",316329,1.32e+00,-6.99e-01
"""rs1000007""",179994,-2.25e-01,5.37e-01


In [29]:
variants = hl.import_table('gs://nbaya/rg_sex/'+phen+'_snps_alleles_N.tsv.gz',types={'N': hl.tint64})
variants = variants.key_by('SNP')
mt_all = hl.read_matrix_table('gs://nbaya/split/ukb31063.hm3_variants.gwas_samples_'+phen+'_grouped_batch_'+batch+'.mt') #matrix table containing individual samples
N = mt_all.count_cols()
variants = variants.annotate(N = hl.int32(N/2))
variants.show()

2018-10-02 13:34:11 Hail: INFO: Reading table with no type imputation
  Loading column 'SNP' as type 'str' (type not specified)
  Loading column 'A1' as type 'str' (type not specified)
  Loading column 'A2' as type 'str' (type not specified)
  Loading column 'N' as type 'int64' (user-specified)

2018-10-02 13:34:16 Hail: INFO: Coerced sorted dataset


+--------------+-----+-----+--------+
| SNP          | A1  | A2  |      N |
+--------------+-----+-----+--------+
| str          | str | str |  int32 |
+--------------+-----+-----+--------+
| "rs1000000"  | "A" | "G" | 180597 |
| "rs10000010" | "C" | "T" | 180597 |
| "rs1000002"  | "T" | "C" | 180597 |
| "rs10000023" | "T" | "G" | 180597 |
| "rs1000003"  | "G" | "A" | 180597 |
| "rs10000033" | "C" | "T" | 180597 |
| "rs10000037" | "A" | "G" | 180597 |
| "rs10000041" | "T" | "G" | 180597 |
| "rs1000007"  | "C" | "T" | 180597 |
| "rs10000075" | "T" | "C" | 180597 |
+--------------+-----+-----+--------+
showing top 10 rows



In [30]:
print(N)

361194


In [9]:
metaA = variants.annotate(Z = ht[variants.SNP].A_Z)
metaB = variants.annotate(Z = ht[variants.SNP].B_Z)

In [10]:
metaB.show()

2018-09-19 20:25:52 Hail: INFO: Coerced sorted dataset
2018-09-19 20:25:54 Hail: INFO: Ordering unsorted dataset with network shuffle


+------------+-----+-----+--------+--------------+
| SNP        | A1  | A2  |      N |            Z |
+------------+-----+-----+--------+--------------+
| str        | str | str |  int64 |      float64 |
+------------+-----+-----+--------+--------------+
| rs1000000  | A   | G   | 168049 | -2.54503e-01 |
| rs10000010 | C   | T   | 168049 |  2.65505e+00 |
| rs1000002  | T   | C   | 168049 |  1.09169e+00 |
| rs10000023 | T   | G   | 168049 |  8.07921e-01 |
| rs1000003  | G   | A   | 168049 | -1.24809e+00 |
| rs10000033 | C   | T   | 168049 | -1.52904e+00 |
| rs10000037 | A   | G   | 168049 |  2.70448e+00 |
| rs10000041 | T   | G   | 168049 | -5.05603e-02 |
| rs1000007  | C   | T   | 168049 | -3.86379e-01 |
| rs10000075 | T   | C   | 168049 |  2.04676e+00 |
+------------+-----+-----+--------+--------------+
showing top 10 rows



In [11]:
metaA.aggregate(hl.agg.mean(metaA.Z))

2018-09-19 20:26:02 Hail: INFO: Coerced sorted dataset
2018-09-19 20:26:04 Hail: INFO: Ordering unsorted dataset with network shuffle


0.002286701004582796

In [12]:
metaB.aggregate(hl.agg.mean(metaB.Z))

2018-09-19 20:27:04 Hail: INFO: Coerced sorted dataset
2018-09-19 20:27:09 Hail: INFO: Ordering unsorted dataset with network shuffle


0.0067514271940972755

In [ ]:
metaA.export('gs://nbaya/rg_sex/'+phen+'_meta_A_s1.tsv.bgz')
metaB.export('gs://nbaya/rg_sex/'+phen+'_meta_B_s1.tsv.bgz')

In [ ]:
ht.export('gs://nbaya/split/meta_split/ukb31063.hm3_'+phen+'_metasplit_final_s3.tsv.bgz')

Check completed sumstats

In [43]:
r_ls = []
for i in range(100):
#     metaA_path = f'gs://nbaya/rg_sex/{phen}_meta_A_n{n_chunks}_batch_{batch}_s{i}.tsv.bgz' 
#     metaB_path = f'gs://nbaya/rg_sex/{phen}_meta_B_n{n_chunks}_batch_{batch}_s{i}.tsv.bgz' 
    metaA_path = f'gs://nbaya/rg_sex/{variant_set}_{phen}_meta_A_n{n_chunks}_constantsexratio_{constant_sex_ratio}_batch_{batch}_s{i}.tsv.bgz'  #first used by constant_sex_ratio=True runs
    metaB_path = f'gs://nbaya/rg_sex/{variant_set}_{phen}_meta_B_n{n_chunks}_constantsexratio_{constant_sex_ratio}_batch_{batch}_s{i}.tsv.bgz'  #first used by constant_sex_ratio=True runs
    metaA = hl.import_table(metaA_path,impute=True,key='SNP')
    metaB = hl.import_table(metaB_path,impute=True,key='SNP')
    both = metaA.rename({'Z':'A_Z'})
    both = both.annotate(B_Z = metaB[both.SNP].Z)
    df = hl.Table.to_pandas(both)
    r = stats.pearsonr(df['A_Z'],df['B_Z'])[0]
    print(r)
    r_ls.append(r)

2019-04-11 12:35:40 Hail: INFO: Reading table to impute column types
2019-04-11 12:35:41 Hail: INFO: Finished type imputation
  Loading column 'SNP' as type 'str' (imputed)
  Loading column 'A1' as type 'str' (imputed)
  Loading column 'A2' as type 'str' (imputed)
  Loading column 'N' as type 'int32' (imputed)
  Loading column 'Z' as type 'float64' (imputed)
2019-04-11 12:35:41 Hail: INFO: Reading table to impute column types
2019-04-11 12:35:42 Hail: INFO: Finished type imputation
  Loading column 'SNP' as type 'str' (imputed)
  Loading column 'A1' as type 'str' (imputed)
  Loading column 'A2' as type 'str' (imputed)
  Loading column 'N' as type 'int32' (imputed)
  Loading column 'Z' as type 'float64' (imputed)
2019-04-11 12:35:43 Hail: INFO: Coerced sorted dataset
2019-04-11 12:35:44 Hail: INFO: Coerced sorted dataset


0.6458892773785383


2019-04-11 12:35:54 Hail: INFO: Reading table to impute column types
2019-04-11 12:35:54 Hail: INFO: Finished type imputation
  Loading column 'SNP' as type 'str' (imputed)
  Loading column 'A1' as type 'str' (imputed)
  Loading column 'A2' as type 'str' (imputed)
  Loading column 'N' as type 'int32' (imputed)
  Loading column 'Z' as type 'float64' (imputed)
2019-04-11 12:35:55 Hail: INFO: Reading table to impute column types
2019-04-11 12:35:56 Hail: INFO: Finished type imputation
  Loading column 'SNP' as type 'str' (imputed)
  Loading column 'A1' as type 'str' (imputed)
  Loading column 'A2' as type 'str' (imputed)
  Loading column 'N' as type 'int32' (imputed)
  Loading column 'Z' as type 'float64' (imputed)
2019-04-11 12:35:57 Hail: INFO: Coerced sorted dataset
2019-04-11 12:35:58 Hail: INFO: Coerced sorted dataset


0.6461672011112516


2019-04-11 12:36:06 Hail: INFO: Reading table to impute column types
2019-04-11 12:36:07 Hail: INFO: Finished type imputation
  Loading column 'SNP' as type 'str' (imputed)
  Loading column 'A1' as type 'str' (imputed)
  Loading column 'A2' as type 'str' (imputed)
  Loading column 'N' as type 'int32' (imputed)
  Loading column 'Z' as type 'float64' (imputed)
2019-04-11 12:36:08 Hail: INFO: Reading table to impute column types
2019-04-11 12:36:08 Hail: INFO: Finished type imputation
  Loading column 'SNP' as type 'str' (imputed)
  Loading column 'A1' as type 'str' (imputed)
  Loading column 'A2' as type 'str' (imputed)
  Loading column 'N' as type 'int32' (imputed)
  Loading column 'Z' as type 'float64' (imputed)
2019-04-11 12:36:09 Hail: INFO: Coerced sorted dataset
2019-04-11 12:36:10 Hail: INFO: Coerced sorted dataset


...

0.6485164419240171


2019-04-11 12:42:18 Hail: INFO: Reading table to impute column types
2019-04-11 12:42:19 Hail: INFO: Finished type imputation
  Loading column 'SNP' as type 'str' (imputed)
  Loading column 'A1' as type 'str' (imputed)
  Loading column 'A2' as type 'str' (imputed)
  Loading column 'N' as type 'int32' (imputed)
  Loading column 'Z' as type 'float64' (imputed)
2019-04-11 12:42:20 Hail: INFO: Reading table to impute column types
2019-04-11 12:42:20 Hail: INFO: Finished type imputation
  Loading column 'SNP' as type 'str' (imputed)
  Loading column 'A1' as type 'str' (imputed)
  Loading column 'A2' as type 'str' (imputed)
  Loading column 'N' as type 'int32' (imputed)
  Loading column 'Z' as type 'float64' (imputed)
2019-04-11 12:42:21 Hail: INFO: Coerced sorted dataset
2019-04-11 12:42:22 Hail: INFO: Coerced sorted dataset


0.6457217776122082


2019-04-11 12:42:32 Hail: INFO: Reading table to impute column types
2019-04-11 12:42:33 Hail: INFO: Finished type imputation
  Loading column 'SNP' as type 'str' (imputed)
  Loading column 'A1' as type 'str' (imputed)
  Loading column 'A2' as type 'str' (imputed)
  Loading column 'N' as type 'int32' (imputed)
  Loading column 'Z' as type 'float64' (imputed)
2019-04-11 12:42:33 Hail: INFO: Reading table to impute column types
2019-04-11 12:42:34 Hail: INFO: Finished type imputation
  Loading column 'SNP' as type 'str' (imputed)
  Loading column 'A1' as type 'str' (imputed)
  Loading column 'A2' as type 'str' (imputed)
  Loading column 'N' as type 'int32' (imputed)
  Loading column 'Z' as type 'float64' (imputed)
2019-04-11 12:42:35 Hail: INFO: Coerced sorted dataset
2019-04-11 12:42:36 Hail: INFO: Coerced sorted dataset


KeyboardInterrupt: 

In [40]:
# old_r_ls = [x[0] for x in r_ls]

In [42]:
print(np.mean(old_r_ls))
print(np.std(old_r_ls))

0.6465881400388376
0.0016638942071025683


In [44]:
new_r_ls = r_ls

In [45]:
print(np.mean(new_r_ls))
print(np.std(new_r_ls))

0.6463037603727204
0.0017053361058531735


(0.6458892773785383, 0.0)